In [50]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

path = r'C:\Users\alber\Instacart Basket Analysis'

In [51]:
customer = pd.read_csv(os.path.join(path, '02 Data','Original Data', 'customers.csv'))

In [52]:
customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


### Renaming columns

In [53]:
customer.rename(columns = {'First Name': 'first_name'}, inplace=True)

In [54]:
customer.rename(columns = {'Surnam':'last_name'}, inplace=True)

In [55]:
customer.rename(columns = {'Gender': 'gender'}, inplace=True)

In [56]:
customer.rename(columns = {'STATE': 'state'}, inplace=True)

In [57]:
customer.rename(columns = {'Age': 'age'}, inplace=True)

In [58]:
customer.rename(columns = {'n_dependants': 'number_of_dependents'}, inplace=True)

In [59]:
customer.rename(columns = {'fam_status': 'marital_status'}, inplace=True)

In [60]:
customer.shape

(206209, 10)

### Consistency check

In [61]:
# Checking for duplicates
customer[customer.duplicated()]

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependents,marital_status,income


In [62]:
# Checking for missing values
customer.isnull().sum()

user_id                     0
first_name              11259
last_name                   0
gender                      0
state                       0
age                         0
date_joined                 0
number_of_dependents        0
marital_status              0
income                      0
dtype: int64

In [63]:
# Creating subset to view missing values
df_nan = customer[customer['first_name'].isnull() == True]

In [64]:
df_nan

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependents,marital_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


In [69]:
# Checking datatypes for all columns
customer.dtypes

user_id                  int64
first_name              object
last_name               object
gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependents     int64
marital_status          object
income                   int64
dtype: object

In [70]:
# Checking for mixed data types
for col in customer.columns.tolist():
  weird = (customer[[col]].applymap(type) != customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customer[weird]) > 0:
    print (col)

first_name


##### Column first_name contains mixed data types.

In [71]:
customer['first_name'].dtype

dtype('O')

#### Column is an object, therefore we convert whole column to be string 

In [72]:
customer['first_name'] = customer['first_name'].astype('str')

In [73]:
# Retest for mixed data types
for col in customer.columns.tolist():
  weird = (customer[[col]].applymap(type) != customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customer[weird]) > 0:
    print (col)

#### Nothing came back, therefore no mixed data types. 

### Combining dataframes

In [75]:
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_new_flags.pkl'))

In [76]:
ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,new_customer,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_prices,spending_flag,median_days_since_previous_order,frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Mid range product,Regularly busy,Regularly busy,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,False,196,1,1,...,Mid range product,Regularly busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,False,196,1,1,...,Mid range product,Regularly busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,False,196,1,1,...,Mid range product,Least busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,False,196,1,1,...,Mid range product,Least busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5,Non-frequent customer


In [78]:
# Checking shape before merge
ords_prods_merged.shape

(32404859, 25)

#### Both dataframes contain user_id. 

In [79]:
# Checking datatype on key column
ords_prods_merged['user_id'].dtypes

dtype('int64')

In [81]:
df_merged = ords_prods_merged.merge(customer, on = 'user_id')

In [83]:
# Checking merge was successful
df_merged.shape

(32404859, 34)

In [85]:
df_merged.drop(columns = ['new_customer'])

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,first_name,last_name,gender,state,age,date_joined,number_of_dependents,marital_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,156685,106143,26,4,23,5.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Gerald,Yates,Male,Hawaii,25,5/26/2017,0,single,53755
32404855,484769,66343,1,6,11,NaN,47210,1,0,Fresh Farmed Tilapia Fillet,...,Non-frequent customer,Jacqueline,Arroyo,Female,Tennessee,22,9/12/2017,3,married,46151
32404856,1561557,66343,2,1,11,30.0,47210,1,1,Fresh Farmed Tilapia Fillet,...,Non-frequent customer,Jacqueline,Arroyo,Female,Tennessee,22,9/12/2017,3,married,46151
32404857,276317,66343,3,6,15,19.0,47210,1,1,Fresh Farmed Tilapia Fillet,...,Non-frequent customer,Jacqueline,Arroyo,Female,Tennessee,22,9/12/2017,3,married,46151


In [86]:
df_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_customer_merge.pkl'))